## OpenAI Function Calling
- 利用可能な関数をLLMに渡して、LLMに「関数を使いたいかどうか」判断をさせる機能
  - LLMが関数を実行するわけではなく、LLMは「関数を使いたいかどうか」を返してくるだけ
- 
- https://platform.openai.com/docs/guides/function-calling

セットアップ

In [2]:
!pip install openai

google colab用

In [ ]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

前準備

In [9]:
from openai import OpenAI
import datetime
import json
client = OpenAI()

Function Calling未対応

In [24]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": "東京の天気は？"}]
  )
response.choices[0].message.content

'ごめんなさい、現在の天気情報は提供できません。最新の天気情報はインターネットや天気アプリをご確認ください。'

Function Calling対応

In [27]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

messages = [{"role": "user", "content": "東京の天気は？"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools, # toolsに利用可能な関数を渡す
    # tool_choiceは
    tool_choice="auto", 
    # tool_choice = {"type": "function", "function": {"name": "get_current_weather"}},
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    messages.append(response_message) # responseをmessageに入れておく(後でChatCompletionする時に必要)

    # responseのtool_calls内にLLMが使用したいと判断した関数とその引数が入っている
    for tool_call in tool_calls:
        # 対象の関数名を関数にマッピング
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        
        # 引数を取得
        function_args = json.loads(tool_call.function.arguments)

        # 関数呼び出し
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # 関数の結果をmessageに入れる
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

    # responseと関数の結果をappendしたmessageで再度リクエスト
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    print(second_response.choices[0].message.content)

東京の現在の天気は10度です。
